In [ ]:
#####財務データのスクレイピング#####
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine
from urllib.request import urlopen
import pandas as pd
import numpy as np
import os
import time
import itertools
import MySQLdb as sql
import mysql.connector
import requests

In [ ]:
##企業データの読み込みとクレンジング
#データの読み込み
company_info1 = pd.read_csv("D:/Statistics/data/stock_price/st-jp/company_info.csv")
index = np.where(np.in1d(company_info1["業種"], np.array(["ETF", "ETN", "REIT", "カントリーF", "インフラF"]))==False)[0]
company_info1 = company_info1.iloc[index]
company_info1.index = np.arange(company_info1.shape[0])
company_info2 = pd.read_csv("D:/Statistics/data/finance_data/stocklist.csv")
company_info2 = company_info2.rename(columns={"銘柄コード": "コード", "市場名": "取引市場", "業種分類": "業種"})
index = np.where(np.in1d(company_info2["業種"], np.array(["REIT銘柄一覧"]))==False)[0]
company_info2 = company_info2.iloc[index]
company_info2.index = np.arange(company_info2.shape[0])

#データの結合
company_info1 = company_info1[["コード", "銘柄名", "取引市場", "業種"]]
company_info2 = company_info2[["コード", "銘柄名", "取引市場", "業種"]]
company_info = pd.concat((company_info1, company_info2), axis=0)
company_info.index = np.arange(company_info.shape[0])

#変数を共通化
company = company_info[["コード", "銘柄名"]].iloc[np.where(company_info[["コード"]].duplicated()==False)[0]]
company_info["銘柄名"] = pd.merge(company_info[["コード"]], company, on="コード", how="left")["銘柄名"]
market = np.array([['水産・農林業', '卸売業', '建設業', '非鉄金属', '鉱業', '機械', 'サービス業', '金属製品',
                      '情報・通信', '食料品', '医薬品', '不動産業', '陸運業', 'その他金融業', '小売業', 'その他製品',
                      '繊維製品', '電気機器', 'ガラス・土石製品', '証券業', '輸送用機器', '石油・石炭製品', '化学',
                      'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行業', '保険業', '倉庫・運輸関連業', '海運業',
                      '空運業', '電気・ガス業'],
                   ['水産・農林', '卸売', '建設業', '非鉄金属', '鉱業', '機械', 'サービス', '金属製品',
                    '情報通信', '食料品', '医薬品', '不動産', '陸運', 'その他金融', '小売', 'その他製品', 
                    '繊維製品', '電気機器', 'ガラス土石', '証券・先物', '輸送用機器', '石油・石炭', '化学',
                    'パルプ・紙', '精密機器', 'ゴム製品', '鉄鋼', '銀行', '保険', '倉庫・運輸', '海運', 
                    '空運', '電気・ガス']])
industry_vec = np.array(company_info["業種"])
for j in range(industry.shape[1]):
    index = np.where(industry_vec==industry[0, j])[0]
    industry_vec[index] = np.array(industry[1, j], dtype="object")
company_info["業種"] = industry_vec
market = np.array([['東証1部', 'JQS', 'JQG', 'マザーズ', '東証2部', '福証Q', '名証2部', '札証', '福証',
                    '名証1部', '札証A', '名証C', '東証'],
                   ['東証1部', '東証JQS', '東証JQG', 'マザーズ', '東証2部', '福岡Q', '名証2部', '札証', "福証",
                    '名証1部',  '札幌ア', '名古屋セ', '東証']])
market_vec = np.array(company_info["取引市場"])
for j in range(market.shape[1]):
    index = np.where(market_vec==market[0, j])[0]
    market_vec[index] = np.array(market[1, j], dtype="object")
company_info["取引市場"] = market_vec

#重複を削除
company_info = company_info.iloc[np.where(company_info["コード"].duplicated()==False)[0]]
company_info.index = np.arange(company_info.shape[0])

In [ ]:
##財務データのスクレイピング
#データの格納用配列
code = np.array(company_info["コード"])
n = company_info.shape[0]
df_list = [i for i in range(n)]

In [ ]:
#証券コードごとにデータを取得
for i in range(180, n):
    print(i)
    
    #urlの読み込みと表の取得
    url = "http://ke.kabupro.jp/xbrl/{code}.htm"
    dfs = pd.read_html(url.format(code=code[i]))
    if len(dfs) < 4:
        continue
        
    #表のクレンジング
    index1 = np.where(dfs[2][0]=="単位百万円")[0]
    index2 = np.where(dfs[2][0].str.contains("データ利用上の注意点"))[0]
    df = dfs[2].iloc[np.arange(index1+1, index2)]
    df.index = np.arange(df.shape[0])
    df.columns = np.array(dfs[2].iloc[index1]).reshape(-1)
    df["company"] = company_info["銘柄名"].iloc[i]
    df["ticker"] = company_info["コード"].iloc[i]
    df["industry"] = company_info["業種"].iloc[i]
    df_list[i] = df

In [ ]:
##財務データのパネルを作成
#取得したデータのインデックス
exist = np.repeat(0, n)
for i in range(n):
    exist[i] = type(df_list[i])!=int
index = np.where(exist==1)[0]
m = index.shape[0]

#リストを変換
ticker_list = [i for i in range(m)]
company_list = [i for i in range(m)]
date_list = [i for i in range(m)]
finance_list = [i for i in range(m)]
for i in range(m):
    ticker_list[i] = np.array(df_list[index[i]]["ticker"])
    company_list[i] = np.array(df_list[index[i]][["company", "industry"]])
    date_list[i] = np.array(df_list[index[i]][["単位百万円", "公開/訂正日"]])
    finance_list[i] = np.array(df_list[index[i]].iloc[:, [1, 2, 3]])
ticker = np.array(list(itertools.chain(*[ticker_list[i] for i in range(m)])))
company = np.array(list(itertools.chain(*[company_list[i] for i in range(m)])))
date = np.array(list(itertools.chain(*[date_list[i] for i in range(m)])))
finance = np.array(list(itertools.chain(*[finance_list[i] for i in range(m)])))
for j in range(finance.shape[1]):
    finance[:, j] = pd.Series(finance[:, j]).str.replace(" ", "").str.replace(",", "")
finance = np.array(finance, dtype="float")    

In [ ]:
#データフレームに変換
ticker_df = pd.DataFrame({"ticker": np.array(ticker, dtype="int")})
company_df = pd.DataFrame({"company": company[:, 0], "industry": company[:, 1]})
date_df = pd.DataFrame({"quarter": date[:, 0], "date": date[:, 1]})
finance_df = pd.DataFrame({"sales": finance[:, 0], "operating_income": finance[:, 1], "net_income": finance[:, 2]})
out_df = pd.concat((ticker_df, company_df, date_df, finance_df), axis=1)

In [ ]:
#データの書き出し
out_df.to_csv("D:/Statistics/data/finance_data.csv", index=None)